# Bioturbation model - calibration using observed data

The bioturbation model relies on an empirical parameter $\beta$, which you may wish to calibrate based on observed data. This notebook shows how you might go about doing that, based on data from this published paper: [Baccaro M, Harrison S, et al 2019](https://doi.org/10.1016/j.envpol.2019.05.106). In this paper, we studied the importance of bioturbation in transporting Ag<sub>2</sub>S nanoparticles (NPs). Full details of this example can be found in the paper.

Say you have the data in [example_data.csv](./example_data.csv), which gives observed concentrations at 7, 14, 21 and 28 days in three soil layers (top, middle, bottom):

In [1]:
import pandas as pd

df = pd.read_csv('./example_data.csv')
df

,day,layer,conc
0,0,top,6.625667
1,0,middle,0.000000
2,0,bottom,0.000000
3,7,top,5.184575
4,7,middle,0.569111
5,7,bottom,0.193135
6,14,top,2.737534
7,14,middle,0.657221
8,14,bottom,0.231144
9,21,top,3.259800


This corresponds to an experimental setup as per the following figure. The three layers where the soil is sample are denoted top, middle and bottom, each of which is 2cm deep. For modelling simplicitly, we conceptualise the soil profile as being six 2-cm layers. 10 mg/kg of Ag<sub>2</sub>S-NPs were added to the top soil layer at day 0.

![Soil profile conceptualisation](../img/calibration-profile.png)

### Creating an optimisable function

We will use SciPy's `leastsq` function to do the optimisation (i.e. finding the $\beta$ value that best fits the data). To do this, we need to create a cost function to pass to the `leastsq` method. To fit multiple layers at once, we flatten all the layers into one array with a length of number of time steps multiplied by the number of layers.

In [4]:
import bioturbation.model as biom
import numpy as np

def f(params, *args):
    """Function to pass to SciPy to optimise"""
    # Get the beta calibration parameter
    beta = params[0]
    # x is the time axis, y is the experimental data, both flattened into a 1D array
    x, y = args[0], args[1]
    # Where to slice the x, y arrays to get individual layers
    n1, n2 = args[2]
    # Initial concentration (top layer only)
    yinit = args[3]
    # Config to pass to the bioturbation model
    config = {
        'n_soil_layers': 6,
        'soil_layer_depth': 0.02,                           # m
        'initial_conc': np.array([yinit, 0, 0, 0, 0, 0]),   # mg/kg dry weight
        'earthworm_density': np.full((6,), 9431.0),         # worms/m3
        'beta': beta
    }
    # Run the bioturbation model
    data = biom.run(**config) 
    # Create empty array for the modelled data and populate
    yfit = np.empty(len(x))
    yfit[:n1] = data[0]         # Top layer
    yfit[n1:n2] = data[3]       # Middle layer
    yfit[n2:] = data[5]         # Bottom layer
    # Return the cost as simply the difference between the modelled and observed
    return y - yfit

### Parsing the experimental data

Now let's parse the experimental data imported above into a format that can be used by the model.

In [6]:
y_obs = np.empty((15,))         # Length of 5 time steps * 3 layers
y_obs[:5] = df[df['layer'] == 'top']['conc'].values
y_obs[5:10] = df[df['layer'] == 'middle']['conc'].values
y_obs[10:] = df[df['layer'] == 'bottom']['conc'].values
x_obs = np.array([0, 7, 14, 21, 28] * 3)

### Fitting the data

Now we can pass these data to SciPy, via the function we defined above, to do the fitting.

In [ ]:
# Initial guess at beta
params0 = np.array(1e-12)
# Obs data
args = (x_obs, y_obs, (5, 10), y_obs[0])
# Run the optimisation
results, cov, info, mesg, ier = leastsq(f, params0, args=args, full_output=True)
beta_optimised = result[0]
print(f'Optimised beta: {beta_optimised}')

# Get the R^2 value of the fit
ss_err = (info['fvec']**2).sum()
ss_tot = ((y_obs - y_obs.mean())**2).sum()
r_squared = 1-(ss_err/ss_tot)
print("R-squared of model fit: {0}".format(r_squared))